In [1]:
import os
import sys
import json

import numpy as np
import pandas as pd

In [2]:
vc_dict = {'acc': 'accommodation_search', 'it': 'job_search_IT', \
           'bank': 'bank_credit', 'wait': 'job_search_waiter_waitress'}

### Setup visual concepts

In [11]:
vc_data = json.load(open('./raw_data/class_scores.json'))

In [12]:
facc = open('./visual_concepts/'+vc_dict['acc']+'.txt', 'a')
fit = open('./visual_concepts/'+vc_dict['it']+'.txt', 'a')
fbank = open('./visual_concepts/'+vc_dict['bank']+'.txt', 'a')
fwait = open('./visual_concepts/'+vc_dict['wait']+'.txt', 'a')

In [13]:
for vsc, situs in vc_data.items():
    facc.write(vsc+' '+situs['acc']+'\n')
    fit.write(vsc+' '+situs['it']+'\n')
    fbank.write(vsc+' '+situs['bank']+'\n')
    fwait.write(vsc+' '+situs['wait']+'\n')
facc.close()
fit.close()
fbank.close()
fwait.close()

### Setup train/val ground_truth user exposure.

In [3]:
gt_train = json.load(open('./raw_data/gt_train.json'))
gt_val = json.load(open('./raw_data/gt_val.json'))
gt_trainval_users = {**gt_train, **gt_val}

In [6]:
gt_train_val_dict = {}
for user, situs in gt_trainval_users.items():
    for situ, score in situs.items():
        if vc_dict[situ] not in gt_train_val_dict:
            gt_train_val_dict[vc_dict[situ]] = {}
        gt_train_val_dict[vc_dict[situ]][user] = score

In [8]:
json.dump(gt_train_val_dict, open('./gt_trainval_uexposure.json', 'w'))

### Setup train/val/test visual concept detection.

In [12]:
cpt_detection_dict = {'train': {}, 'val': {}, 'test': {}}
pred_train = json.load(open('./raw_data/predictions_train.json'))
pred_val = json.load(open('./raw_data/predictions_val.json'))
pred_test = json.load(open('./raw_data/predictions_test.json'))

In [16]:
for user, ims in pred_train.items():
    cpt_detection_dict['train'][user] = {}
    for im, viscpts in ims.items():
        cpt_detection_dict['train'][user][im] = {}
        for cpt_det in viscpts:
            parts = cpt_det.split(' ')
            cpt_name = parts[0]
            cpt_score = float(parts[1])
            if cpt_name not in cpt_detection_dict['train'][user][im]:
                cpt_detection_dict['train'][user][im][cpt_name] = []
            cpt_detection_dict['train'][user][im][cpt_name].append(cpt_score)
            
for user, ims in pred_val.items():
    cpt_detection_dict['val'][user] = {}
    for im, viscpts in ims.items():
        cpt_detection_dict['val'][user][im] = {}
        for cpt_det in viscpts:
            parts = cpt_det.split(' ')
            cpt_name = parts[0]
            cpt_score = float(parts[1])
            if cpt_name not in cpt_detection_dict['val'][user][im]:
                cpt_detection_dict['val'][user][im][cpt_name] = []
            cpt_detection_dict['val'][user][im][cpt_name].append(cpt_score)
            
for user, ims in pred_test.items():
    cpt_detection_dict['test'][user] = {}
    for im, viscpts in ims.items():
        cpt_detection_dict['test'][user][im] = {}
        for cpt_det in viscpts:
            parts = cpt_det.split(' ')
            cpt_name = parts[0]
            cpt_score = float(parts[1])
            if cpt_name not in cpt_detection_dict['test'][user][im]:
                cpt_detection_dict['test'][user][im][cpt_name] = []
            cpt_detection_dict['test'][user][im][cpt_name].append(cpt_score)

In [17]:
json.dump(cpt_detection_dict, open('./trainvaltest_detection_split_imageclef.json', 'w'))